EP02.1 - Visão e Processamento de Imagens
===

Nº USP | Nome
---|---|
11347794|Aarão Melo Lopes|

---

## Acesso


Se estiver utilizando o Google Colaboratory execute os passos a seguir. No entanto, se estiver rodando em um dispositivo local, ignore esta seção.


Realize login com sua conta google, para ter acesso aos arquivos compartilhados, executando a célula abaixo.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Mude o diretório corrente para a pasta do projeto.

In [10]:
cd drive/Shared drives/EP02 - Visão e Processamento de Imagens

/content/drive/Shared drives/EP02 - Visão e Processamento de Imagens


## Geração



Carregue as imagens do diretório `Dataset` em uma matriz.

In [6]:
from skimage.io import imread, imshow, imsave

def loadDir(path):
  dataset = []
  for i in range(1080):
    dataset.append(imread(path+"/" + str(i+1) + '.jpeg'))
    print('read ' + str(i+1) + '.jpeg')
  return dataset

def saveDir(dataset, path):
  for i in range(1080):
    imsave(path + '/'+str(i+1) + '.jpeg', dataset[i])
    print('save ' + str(i+1) + '.jpeg')



In [ ]:
dataset = loadDir("Dataset")

In [ ]:
pip install imgaug

In [ ]:
mkdir augmentedDataset

### Escala de Cinza

In [ ]:
mkdir augmentedDataset/grayScale

Execute a célula abaixo para criar e salvar as images em escala de cinza

In [ ]:
import imgaug.augmenters as iaa
grayScale = iaa.Grayscale(alpha=1.0).augment_images(dataset)
saveDir(grayScale, 'augmentedDataset/grayScale')

In [ ]:
greyScale = loadDir("augmentedDataset/grayScale")

In [27]:
width  = 720
height = 1280
gradient = [[0 for j in range(width)] for i in range(height)]
for i in range(height):
  for j in range(width):
    x = (j+0.5)/width-0.5
    y = (i+0.5)/height-0.5
    color =  int(((x**2+y**2)**0.5/(2**0.5))**0.7*256)
    gradient[i][j] = [color, color, color]

In [ ]:
from skimage.util import img_as_ubyte
import cv2
shadow = []
for i in range(1080):
  shadow.append(cv2.add(np.uint8(gradient),img_as_ubyte(greyScale[i])))

In [23]:
mkdir augmentedDataset/shadowScale

In [ ]:
saveDir(shadow, "augmentedDataset/shadowScale")

## Visualização


Visualização dos dados aumentados

In [18]:
import matplotlib.pyplot as plt
def showimages(image_list):
  for j in range(40):
    fig, axes = plt.subplots(1, 27, figsize=(20, 20))
    ax = axes.ravel()
    for i in range(27):
        ax[i].imshow(image_list[i+j*27])
        ax[i].axis('off')
    fig.tight_layout()
    plt.show()

In [ ]:
dataset = loadDir("Dataset")

In [ ]:
showimages(shadow)

In [ ]:
showimages(dataset)